In [ ]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json
from datetime import datetime
from collections import OrderedDict

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the database to a variable name
db = mongo.chicago_bikes

In [ ]:
#define collection variables
divvy_rides = db['divvy_ridedata']
weather_daily = db['weather_daily']
divvy_ridedata_merged = db['divvy_ridedata_merged']

In [ ]:
divvy_ridedata_merged = db["divvy_ridedata_merged"]

# Define the aggregation pipeline
pipeline = [
    {
        "$group": {
            "_id": {
                "year": {"$year": {"$toDate": "$started_at"}},
                "month": {"$month": {"$toDate": "$started_at"}}
            },
            "total_rides": {"$sum": 1}
        }
    },
    {
        "$project": {
            "_id": 0,
            "year": "$_id.year",
            "month": "$_id.month",
            "total_rides": 1
        }
    },
    {
        "$sort": {"year": 1, "month": 1}
    }
]

# Execute the aggregation pipeline and write to a new collection
divvy_rides_by_month = db["divvy_rides_by_month"]
divvy_rides_by_month.drop()  # Drop the collection
aggregated_result = divvy_ridedata_merged.aggregate(pipeline, allowDiskUse=True, collation=None)

for doc in aggregated_result:
    divvy_rides_by_month.insert_one(doc)

print("Aggregation result has been written to the new collection.")

In [ ]:
print(db.list_collection_names())

In [ ]:
# Define the projection to include the "month" field and exclude the "_id" field
projection = {"month": 1, "_id": 0}

# Retrieve the first document using find_one() with the projection
result = divvy_rides_by_month.find_one({}, projection)

# Print the result
print(result)


In [ ]:
# Find all documents in the collection
all_documents = divvy_rides_by_month.find({})

# Print each document
for document in all_documents:
    print(document)

In [ ]:
divvy_rides_by_month = db["divvy_rides_by_month"]

# Define the aggregation pipeline
pipeline = [
    {
        "$group": {
            "_id": {
                "year": "$year",
                "season": {
                    "$switch": {
                        "branches": [
                            {"case": {"$in": ["$month", [3, 4, 5]]}, "then": "Spring"},
                            {"case": {"$in": ["$month", [6, 7, 8]]}, "then": "Summer"},
                            {"case": {"$in": ["$month", [9, 10, 11]]}, "then": "Autumn"},
                            {"case": {"$in": ["$month", [12, 1, 2]]}, "then": "Winter"}
                        ],
                        "default": "Unknown"
                    }
                }
            },
            "total_rides": {"$sum": "$total_rides"}
        }
    },
    {
        "$sort": {"_id.year": 1, "_id.season": 1}
    }
]

# Execute the aggregation pipeline
divvy_rides_by_season = db["divvy_rides_by_season"]
divvy_rides_by_season.drop()  # Drop the collection
aggregated_result = list(divvy_rides_by_month.aggregate(pipeline, allowDiskUse=True, collation=None))

# Insert the aggregated documents into the new collection
for doc in aggregated_result:
    print("Inserting document:", doc)
    divvy_rides_by_season.insert_one(doc)

print("Aggregation by season result has been written to the new collection.")

In [ ]:
# Find all documents in the collection
all_documents = divvy_rides_by_season.find({})

# Print each document
for document in all_documents:
    print(document)

In [ ]:
Top10StartStations = db['Top10StartStations'].find()
db.Top10StartStations.find_one()

In [ ]:
db.Top10StartStations.find_one()